In [1]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQtBXo5cBnDsM2fmfHPm6u72KGUS5FjPHNGMxOfYjA9-CAhmnRpwkIw_rOR3sANJIToiUU__6fbBvig/pub?gid=572763137&single=true&output=csv")


,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


In [9]:
df.shape
df['Pass/Fail'].value_counts()

-1    1463
 1     104
Name: Pass/Fail, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df=df.drop('Time',axis=1)

In [13]:
for column in df.columns:
    min_value=df[column].min()
    max_value=df[column].max()
    
    # Generate random numbers within the range
    random_values = np.random.uniform(min_value, max_value, size=df[column].isnull().sum())
    
    # Create a Series with the random values
    random_series = pd.Series(random_values, index=df[column][df[column].isnull()].index)

    # Fill NaN values with the random series
    df[column].fillna(random_series, inplace=True)

In [24]:
# Separate features and target

X=df.drop('Pass/Fail',axis=1)
y=df['Pass/Fail']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape)
print(X_test.shape)

lr=LogisticRegression(max_iter=10000)
lr.fit(X_train,y_train)

y_pred=lr.predict(X_test)

accuracy=accuracy_score(y_test,y_pred)

print(accuracy)

(1253, 590)
(314, 590)
0.89171974522293


C:\Users\Lucky Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Now apply various Filter based techniques 

In [67]:


df=pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQtBXo5cBnDsM2fmfHPm6u72KGUS5FjPHNGMxOfYjA9-CAhmnRpwkIw_rOR3sANJIToiUU__6fbBvig/pub?gid=572763137&single=true&output=csv")


In [68]:
#1 Duplicate removal


print('Original columns of the data was:',df.shape[1])
df.drop('Time',inplace=True,axis=1)


new_df=df.T.drop_duplicates().T
print('After removing duplicates total columsn:',new_df.shape[1])

Original columns of the data was: 592
After removing duplicates total columsn: 487


In [69]:
#2 Variance Threshold
from sklearn.feature_selection import VarianceThreshold
from scipy.stats import pearsonr

sel=VarianceThreshold(threshold=0.05)
sel.fit(new_df)

columns=new_df.columns[sel.get_support()]

new_df=sel.fit_transform(new_df)

new_df=pd.DataFrame(new_df,columns=columns)

print('After VT now remaining columns are:',new_df.shape[1])

After VT now remaining columns are: 286


In [75]:
#3 Correaltiom

corr_matrix=new_df.corr()
columns=corr_matrix.columns
columns_to_drop=[]

for i in range(len(columns)):
    for j in range(i+1,len(columns)):
        if corr_matrix.loc[columns[i],columns[j]]>0.8:
            columns_to_drop.append(columns[j])
            
print(len(columns_to_drop))

columns_to_drop=set(columns_to_drop)
len(columns_to_drop)

new_df_corr=new_df.drop(columns=columns_to_drop,axis=1)
print("After correlation remaining columns are:",new_df_corr.shape[1])

408
After correlation remaining columns are: 153


In [79]:
#We can still perform ANOVA analysis by considering each column separately.

#ANOVA can be applied to each column individually, comparing the target variable which is Pass/Fail column in our case against the non-missing values in that specific column.

#We will drop NaN values of each column before performing the ANOVA

In [80]:
new_df_corr

,0,1,2,3,4,6,12,14,15,16,...,562,564,569,570,571,572,573,581,585,Pass/Fail
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,97.6133,202.4396,7.9558,414.8710,10.0433,...,NaN,NaN,NaN,533.8500,2.1113,8.95,0.3157,NaN,2.3630,-1.0
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,102.3433,200.5470,10.1548,414.7347,9.2599,...,NaN,NaN,NaN,535.0164,2.4335,5.92,0.2653,208.2045,4.4447,-1.0
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,95.4878,202.0179,9.5157,416.7075,9.3144,...,267.064,1.10,68.8489,535.0245,2.0293,11.21,0.1882,82.8602,3.1745,1.0
3,2988.72,2479.90,2199.0333,909.7926,1.3204,104.2367,201.8482,9.6052,422.2894,9.6924,...,268.228,7.32,25.0363,530.5682,2.0253,9.33,0.1738,73.8432,2.0544,-1.0
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.3967,201.9424,10.5661,420.5925,10.3387,...,NaN,NaN,NaN,532.0155,2.0275,8.83,0.2224,NaN,99.3032,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2899.41,2464.36,2179.7333,3085.3781,1.4843,82.2467,203.9867,11.7692,419.3404,10.2397,...,264.272,4.98,15.4662,536.3418,2.0153,7.98,0.2363,203.1720,2.8669,-1.0
1563,3052.31,2522.55,2198.5667,1124.6595,0.8763,98.4689,204.0173,9.1620,405.8178,10.2285,...,266.832,4.56,20.9118,537.9264,2.1814,5.48,0.3891,NaN,2.6238,-1.0
1564,2978.81,2379.78,2206.3000,1110.4967,0.8236,99.4122,NaN,NaN,NaN,NaN,...,256.730,11.09,29.0954,530.3709,2.3435,6.49,0.4154,43.5231,3.0590,-1.0
1565,2894.92,2532.01,2177.0333,1183.7287,1.5726,98.7978,197.2448,9.7354,401.9153,9.8630,...,264.272,4.98,15.4662,534.3936,1.9098,9.13,0.3669,93.4941,3.5662,-1.0


In [108]:
from scipy.stats import f_oneway
alpha=0.05

# Columns haveing p_value less than alpha
column_pvalues = []

for column in new_df_corr.iloc[:,:-1].columns:
    column_data=new_df_corr[column].dropna()
    
    # Perform ANOVA with the target variable
    anova_result=f_oneway(column_data,new_df_corr['Pass/Fail'])
    
    # Print the ANOVA result or perform further analysis
    #print(f"Column: {column} - ANOVA p-value: {anova_result.pvalue}")
    
    if anova_result.pvalue<=alpha:
        column_pvalues.append((column,anova_result.pvalue))
# Selecting best 100 Features - lower p-value better feature
# Sort the column p-values in ascending order
column_pvalues.sort(key=lambda x:x[1])

selected_column=[column for column,_ in column_pvalues[:100]]
data_anova = new_df_corr[selected_column+['Pass/Fail']]
data_anova
print("Number of Columns after Correlation- ", data_anova.shape[1])

Number of Columns after Correlation-  101


In [114]:
# Shape of Data after Feature Selection

print("Shape - ", data_anova.shape)

# Filling NaN Values with random 
for column in data_anova.columns:

    # Get the minimum and maximum values of the column
    min_value = data_anova[column].min()
    max_value = data_anova[column].max()

    # Generate random numbers within the range
    random_values = np.random.uniform(min_value, max_value, size=data_anova[column].isnull().sum())
    
     # Create a Series with the random values
    random_series = pd.Series(random_values, index=data_anova[column][data_anova[column].isnull()].index)
    
        # Fill NaN values with the random series
    data_anova[column].fillna(random_series, inplace=True)
    
# Separate features and target
X=data_anova.drop('Pass/Fail',axis=1)
y=data_anova['Pass/Fail']

# Split data into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape)
print(X_test.shape)

# Initialize and train logistic regression model
log_reg=LogisticRegression(max_iter=10000)
log_reg.fit(X_train,y_train)

# Make predictions on the test set
y_pred=log_reg.predict(X_test)

accuracy=accuracy_score(y_test,y_pred)
print("Test accuracy is:",accuracy)

Shape -  (1567, 101)
(1253, 100)
(314, 100)


C:\Users\Lucky Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Test accuracy is: 0.9076433121019108


C:\Users\Lucky Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
